# Dealing with Timestamps in SQLite

**The date and time are often stored in database when a record is created or updated.**

**For this exercise, you need to build an `Account` class that creates a personal bank account and makes transactions (deposit or withdrawal). Create a database with tables to store the account details and transaction details for each user.**

In [1]:
import sqlite3

In [2]:
class Account(object):
    
    def __init__(self, name, opening_balance=0.0):
        self.name = name
        self._balance = opening_balance
        print(f"Account created for {self.name}")
        self.show_balance()
    
    def deposit(self, amount):
        if amount > 0.0:
            self._balance += amount
            print(f"£{amount} deposited")
        return self._balance
    
    def withdraw(self, amount):
        if 0.0 < amount <= self._balance:
            self._balance -= amount
            print(f"£{amount} withdrawn - £{self._balance} left")
            return amount
        else:
            print(f"Not enough funds to withdraw £{amount}")
            return 0.0
    
    def show_balance(self):
        print(f"Balance for {self.name} is £{self._balance}")


In [3]:
john = Account('John', 100000)

john.deposit(10.10)
john.deposit(102.34)

john.withdraw(20000)
john.withdraw(150000)

john.show_balance()

Account created for John
Balance for John is £100000
£10.1 deposited
£102.34 deposited
£20000 withdrawn - £80112.44 left
Not enough funds to withdraw £150000
Balance for John is £80112.44


**ALTERNATIVELY, the more accepted form and standard protocol in financial databases is to store the decimal values as objects to be retrieved:**

In [4]:
from decimal import * 


class Account(object):
    _qb = Decimal('0.00')  # Class constant, accessible without creating an instance
    
    def __init__(self, name: str, opening_balance: float = 0.0):
        self.name = name
        # Convert float value to decimal object
        self._balance = Decimal(opening_balance).quantize(Account._qb)
        print("\tAccount created for {}. ".format(self.name), end='')
        self.show_balance()
    
    def deposit(self, amount: float) -> Decimal:
        decimal_amount = Decimal(amount).quantize(Account._qb)
        if decimal_amount > Account._qb:
            self._balance = self._balance + decimal_amount
            print("£{} deposited".format(decimal_amount))
        return self._balance
    
    def withdraw(self, amount: float) -> Decimal:
        decimal_amount = Decimal(amount).quantize(Account._qb)
        if Account._qb < decimal_amount <= self._balance:
            self._balance = self._balance - decimal_amount
            print("£{} withdrawn".format(decimal_amount))
            return decimal_amount
        else:
            print("There are not enough funds in your account")
            return Account._qb
    
    def show_balance(self):
        print("Balance on {}'s account is £{}".format(self.name, self._balance))


In [5]:
if __name__ == '__main__':
    tim = Account("Tim")
    tim.deposit(10.1)
    tim.deposit(0.1)
    tim.deposit(0.1)
    tim.withdraw(0.3)
    tim.withdraw(0)
    tim.show_balance()
    
    print("=" * 80)
    x = tim.withdraw(900)
    print(str(x) + " withdrawn")
    tim.show_balance()

	Account created for Tim. Balance on Tim's account is £0.00
£10.10 deposited
£0.10 deposited
£0.10 deposited
£0.30 withdrawn
There are not enough funds in your account
Balance on Tim's account is £10.00
There are not enough funds in your account
0.00 withdrawn
Balance on Tim's account is £10.00


**The `Decimal` class creates a decimal object from the float values. Even though the client works with normal float values, the values are stored as decimal objects.**

**Now that you have a class to create and manage bank accounts, you need to connect with the database that stores the account information. There are two tables, one for `accounts` (name and balance) and another for `transactions` (time of transaction, account name, and amount that is positive for a deposit and negative for a withdrawal).**

**The primary key in accounts is the account holder's name, e.g. 'Tim'.**

**The primary keys (composite key) in transactions is a mixture of time of transaction and account name, i.e. both together are unique. This is based on the logic that two Tims cannot withdraw money at the exact same second. In the real world, each account would be assigned a unique ID.**

**The `Accounts` class creates the account and adds it to the `accounts` table, then the class instance runs various transactions, which are stored in the `transactions` table.**

**NOTE: You need to update the `Accounts` class `__init__` method to make sure that during class instance creation, it connects to the database and saves the information. You also need to update the `deposit()` and `withdraw()` methods to store transaction details in database.**

In [6]:
db = sqlite3.connect('accounts/personal_accounts.sqlite')

db.execute("CREATE TABLE IF NOT EXISTS accounts (name TEXT PRIMARY KEY NOT NULL, balance INTEGER NOT NULL)")
db.execute("CREATE TABLE IF NOT EXISTS transactions (time TIMESTAMP NOT NULL, account TEXT NOT NULL, "
           "amount INTEGER NOT NULL, PRIMARY KEY (time, account))")

In [7]:
# UPDATE 'init' METHOD

class Account(object):
    
    def __init__(self, name, opening_balance=0):
        cursor = db.execute("SELECT name, balance FROM accounts WHERE (name=?)", (name, ))
        row = cursor.fetchone()
        
        if row:
            # Retrieve existing account
            self.name, self._balance = row
            print(f"Retrieved account details for {self.name}")
        else:
            # Create new account
            self.name = name
            self._balance = opening_balance
            cursor.execute("INSERT INTO accounts VALUES(?, ?)", (name, opening_balance))
            cursor.connection.commit()
            print(f"Account created for {self.name}")
            
        self.show_balance()
    
    def deposit(self, amount):
        if amount > 0.0:
            self._balance += amount
            print("£{:.2f} deposited".format(amount / 100))
            
        return self._balance / 100
    
    def withdraw(self, amount):
        if 0.0 < amount <= self._balance:
            self._balance -= amount
            print("£{:.2f} withdrawn".format(amount / 100))
            return amount / 100
        else:
            print(f"Not enough funds to withdraw £{amount}")
            return 0.0
    
    def show_balance(self):
        print("Balance on account is £{:.2f}".format(self._balance / 100))


In [8]:
# Add accounts to database

john = Account('John', 100000)
terryJ = Account('TerryJ')
graham = Account('Graham', 1000)
sally = Account('Sally', 700)
terryG = Account('TerryG')

Account created for John
Balance on account is £1000.00
Account created for Terry
Balance on account is £0.00
Account created for Graham
Balance on account is £10.00
Account created for Sally
Balance on account is £7.00


In [10]:
for row in db.execute("SELECT * FROM accounts"):
    print(row)

('John', 100000)
('Terry', 0)
('Graham', 1000)
('Sally', 700)


In [11]:
db.close()

In [16]:
# UPDATE CLASS METHODS

import datetime
import pytz

class Account(object):
    
    @staticmethod
    def _current_time():
        return pytz.utc.localize(datetime.datetime.utcnow())
    
    def __init__(self, name, opening_balance=0):
        cursor = db.execute("SELECT name, balance FROM accounts WHERE (name=?)", (name, ))
        row = cursor.fetchone()
        
        if row:
            self.name, self._balance = row
            print(f"\tRetrieved account details for {self.name}")
        else:
            self.name = name
            self._balance = opening_balance
            cursor.execute("INSERT INTO accounts VALUES(?, ?)", (name, opening_balance))
            cursor.connection.commit()
            print(f"\tAccount created for {self.name}")
            
        self.show_balance()
    
    # Update balance in amounts and add transaction to transactions
    
    def deposit(self, amount):
        if amount > 0.0:
            new_balance = self._balance + amount
            deposit_time = Account._current_time()
            db.execute("UPDATE accounts SET balance= ? WHERE (name=?)", (new_balance, self.name))
            db.execute("INSERT INTO transactions VALUES(?, ?, ?)", (deposit_time, self.name, amount))
            db.commit()
            self._balance = new_balance
            print("£{:.2f} deposited".format(amount / 100))
            
        return self._balance / 100
    
    # Update balance in amounts and add transaction to transactions
    
    def withdraw(self, amount):
        if 0.0 < amount <= self._balance:
            new_balance = self._balance - amount
            withdrawal_time = Account._current_time()
            db.execute("UPDATE accounts SET balance = ? WHERE (name=?)", (new_balance, self.name))
            db.execute("INSERT INTO transactions VALUES(?, ?, ?)", (withdrawal_time, self.name, -amount))
            db.commit()
            self._balance = new_balance
            print("£{:.2f} withdrawn".format(amount / 100))
            return amount / 100
        else:
            print(f"Not enough funds to withdraw £{amount}")
            return 0.0
    
    def show_balance(self):
        print("Balance on account is £{:.2f}".format(self._balance / 100))



**Retrieve each account (and add two new ones) and make some transactions:**

In [13]:
db = sqlite3.connect('accounts/personal_accounts.sqlite')

# Check accounts are there
for row in db.execute("SELECT * FROM accounts"):
    print(row)

('John', 100000)
('Terry', 0)
('Graham', 1000)
('Sally', 700)


In [17]:
john = Account('John')
john.deposit(2002)
john.withdraw(1010)
john.show_balance()

print('*' * 80)

terry = Account('Terry')
terry.deposit(1)
terry.deposit(1)
terry.deposit(1)
terry.show_balance()

print('*' * 80)

# New account
terryJ = Account('TerryJ')
terryJ.deposit(100)
terryJ.deposit(5)
terryJ.show_balance()

print('*' * 80)

graham = Account('Graham')
graham.withdraw(100)
graham.show_balance()

print('*' * 80)

sally = Account('Sally')
sally.deposit(1000)
sally.show_balance()

print('*' * 80)

# New account
terryG = Account('TerryG')
terryG.deposit(4000)
terryG.withdraw(5000)
terryG.show_balance()

print('*' * 80)

	Retrieved account details for John
Balance on account is £1010.02
£20.02 deposited
£10.10 withdrawn
Balance on account is £1019.94
********************************************************************************
	Retrieved account details for Terry
Balance on account is £0.03
£0.01 deposited
£0.01 deposited
£0.01 deposited
Balance on account is £0.06
********************************************************************************
	Retrieved account details for TerryJ
Balance on account is £2.06
£1.00 deposited
£0.05 deposited
Balance on account is £3.10
********************************************************************************
	Retrieved account details for Graham
Balance on account is £8.00
£1.00 withdrawn
Balance on account is £7.00
********************************************************************************
	Retrieved account details for Sally
Balance on account is £27.00
£10.00 deposited
Balance on account is £37.00
*******************************************************

In [18]:
for row in db.execute("SELECT * FROM transactions"):
    print(row)

('2024-03-10 09:03:06.513662+00:00', 'John', 20.02)
('2024-03-10 09:03:06.520644+00:00', 'John', -10.1)
('2024-03-10 09:03:06.525631+00:00', 'Terry', 0.1)
('2024-03-10 09:03:06.531613+00:00', 'Terry', 0.1)
('2024-03-10 09:03:06.538595+00:00', 'Terry', 0.1)
('2024-03-10 09:03:06.549566+00:00', 'TerryJ', 100)
('2024-03-10 09:03:06.554552+00:00', 'TerryJ', 0.5)
('2024-03-10 09:03:06.560541+00:00', 'Graham', -100)
('2024-03-10 09:03:06.565524+00:00', 'Sally', 1000)
('2024-03-10 09:03:06.576495+00:00', 'TerryG', 40)
('2024-03-10 09:08:04.912219+00:00', 'John', 2002)
('2024-03-10 09:08:04.920574+00:00', 'John', -1010)
('2024-03-10 09:08:04.927349+00:00', 'Terry', 1)
('2024-03-10 09:08:04.933891+00:00', 'Terry', 1)
('2024-03-10 09:08:04.941149+00:00', 'Terry', 1)
('2024-03-10 09:08:04.947628+00:00', 'TerryJ', 100)
('2024-03-10 09:08:04.954618+00:00', 'TerryJ', 5)
('2024-03-10 09:08:04.960940+00:00', 'Graham', -100)
('2024-03-10 09:08:04.967625+00:00', 'Sally', 1000)
('2024-03-10 09:08:04.9744

In [19]:
for row in db.execute("SELECT * FROM accounts"):
    print(row)

('John', 101993.92)
('Terry', 6.3)
('Graham', 700)
('Sally', 3700)
('TerryJ', 310.5)
('TerryG', 3040)


In [20]:
db.close()

**There is identical code in both the `deposit()` and `withdraw()` method:**

    new_balance = self._balance + amount
    deposit_time = Account._current_time()
    db.execute("UPDATE accounts SET balance= ? WHERE (name=?)", (new_balance, self.name))
    db.execute("INSERT INTO transactions VALUES(?, ?, ?)", (deposit_time, self.name, amount))
    db.commit()
    self._balance = new_balance
    
**This common code can be put in its own function, to be called by both transaction methods:**

    def _save_update(self, amount):
        new_balance = self._balance + amount
        deposit_time = Account._current_time()
        db.execute("UPDATE accounts SET balance= ? WHERE (name=?)", (new_balance, self.name))
        db.execute("INSERT INTO transactions VALUES(?, ?, ?)", (deposit_time, self.name, amount))
        db.commit()
        self._balance = new_balance

**The `_save_update()` method can be called to store any transaction in the database table (leading underscore in the method name to show that it should not be used outside of the `Accounts` class).**

**Basically, you now have a database with some accounts and some transactions data, where one of the primary keys is a datetime type.**

## Dealing with datatypes

**The `transactions` table is essentially a 'history' of the movements of an account, with the UTC timestamp as the unique feature. You may ask why not just select the last transaction on the account in the table and delete it, but you cannnot find the latest date and time value for an account if the datetime values are stored as strings.**

In [3]:
class Account(object):
    
    @staticmethod
    def _current_time():
        return pytz.utc.localize(datetime.datetime.utcnow())
    
    def __init__(self, name, opening_balance=0):
        cursor = db.execute("SELECT name, balance FROM accounts WHERE (name=?)", (name, ))
        row = cursor.fetchone()
        
        if row:
            self.name, self._balance = row
            print(f"\tRetrieved account details for {self.name}")
        else:
            self.name = name
            self._balance = opening_balance
            cursor.execute("INSERT INTO accounts VALUES(?, ?)", (name, opening_balance))
            cursor.connection.commit()
            print(f"\tAccount created for {self.name}")
            
        self.show_balance()
    
    def _save_update(self, amount):
        new_balance = self._balance + amount
        deposit_time = Account._current_time()
        db.execute("UPDATE accounts SET balance= ? WHERE (name=?)", (new_balance, self.name))
        db.execute("INSERT INTO transactions VALUES(?, ?, ?)", (deposit_time, self.name, amount))
        db.commit()
        self._balance = new_balance
    
    def deposit(self, amount):
        if amount > 0.0:
            # Call new update method to store data in database
            self._save_update(amount)
            print("£{:.2f} deposited".format(amount / 100))
            
        return self._balance / 100
    
    def withdraw(self, amount):
        if 0.0 < amount <= self._balance:
            # Call new update method to store data in database
            self._save_update(-amount)
            print("£{:.2f} withdrawn".format(amount / 100))
            return amount / 100
        else:
            print(f"Not enough funds to withdraw £{amount}")
            return 0.0
    
    def show_balance(self):
        print("Balance on account is £{:.2f}".format(self._balance / 100))



In [4]:
# Unique feature of a transaction is the timestamp

db = sqlite3.connect('accounts/personal_accounts.sqlite')

for row in db.execute("SELECT * FROM transactions WHERE account='Graham'"):
    print(row)

('2024-03-10 09:03:06.560541+00:00', 'Graham', -100)
('2024-03-10 09:08:04.960940+00:00', 'Graham', -100)
('2024-03-10 09:08:17.975260+00:00', 'Graham', -100)


In [5]:
for row in db.execute("SELECT * FROM transactions WHERE account='Graham'"):
    local_time = row[0]
    print(f"{local_time}\t{type(local_time)}")

2024-03-10 09:03:06.560541+00:00	<class 'str'>
2024-03-10 09:08:04.960940+00:00	<class 'str'>
2024-03-10 09:08:17.975260+00:00	<class 'str'>


In [6]:
db.close()

**Note that the timestamp is retrieved as a string, which is not what you want. You could convert the values with Python `datetime` module, or you can get SQLite to 'respond' to the string values as registered timestamp datatype, by passing `PARSE_DECLTYPES` when connecting to the database.**

**NOTE: It does not handle *timezone-aware* date objects, i.e. you won't get an offset value between local time and UTC time. If this is essential to your work, there are other libraries that can handle timezone-aware dates, like python dateutil or Arrow.**

In [7]:
import datetime
import pytz

db = sqlite3.connect('accounts/personal_accounts.sqlite', detect_types=sqlite3.PARSE_DECLTYPES)

In [8]:
for row in db.execute("SELECT * FROM transactions WHERE account='Graham'"):
    print(row)

(datetime.datetime(2024, 3, 10, 9, 3, 6, 560541), 'Graham', -100)
(datetime.datetime(2024, 3, 10, 9, 8, 4, 960940), 'Graham', -100)
(datetime.datetime(2024, 3, 10, 9, 8, 17, 975260), 'Graham', -100)


In [9]:
for row in db.execute("SELECT * FROM transactions WHERE account='Graham'"):
    local_time = row[0]
    print(f"{local_time}\t{type(local_time)}")

2024-03-10 09:03:06.560541	<class 'datetime.datetime'>
2024-03-10 09:08:04.960940	<class 'datetime.datetime'>
2024-03-10 09:08:17.975260	<class 'datetime.datetime'>


## Dealing with timezones

**There are two different ways to display local times:**

 - **Convert UTC time to local time using `pytz` and `datetime` modules**
 - **Convert UTC time to local time using SQLite datetime `strftime()` function. This has the advantage of working no matter what is used to access the database.**

In [10]:
# Offset now on display

for row in db.execute("SELECT * FROM transactions WHERE account='Graham'"):
    utc_time = row[0]
    local_time = pytz.utc.localize(utc_time).astimezone()
    print(f"{local_time}\t{type(local_time)}")

2024-03-10 09:03:06.560541+00:00	<class 'datetime.datetime'>
2024-03-10 09:08:04.960940+00:00	<class 'datetime.datetime'>
2024-03-10 09:08:17.975260+00:00	<class 'datetime.datetime'>


In [11]:
# Local times using SQLite

for row in db.execute("SELECT strftime('%Y-%m-%d %H:%M:%f', transactions.time, 'localtime') AS localtime, transactions.account, "
                      "transactions.amount FROM transactions ORDER BY transactions.time"):
    print(row)

('2024-03-10 09:03:06.514', 'John', 20.02)
('2024-03-10 09:03:06.521', 'John', -10.1)
('2024-03-10 09:03:06.526', 'Terry', 0.1)
('2024-03-10 09:03:06.532', 'Terry', 0.1)
('2024-03-10 09:03:06.539', 'Terry', 0.1)
('2024-03-10 09:03:06.550', 'TerryJ', 100)
('2024-03-10 09:03:06.555', 'TerryJ', 0.5)
('2024-03-10 09:03:06.561', 'Graham', -100)
('2024-03-10 09:03:06.566', 'Sally', 1000)
('2024-03-10 09:03:06.576', 'TerryG', 40)
('2024-03-10 09:08:04.912', 'John', 2002)
('2024-03-10 09:08:04.921', 'John', -1010)
('2024-03-10 09:08:04.927', 'Terry', 1)
('2024-03-10 09:08:04.934', 'Terry', 1)
('2024-03-10 09:08:04.941', 'Terry', 1)
('2024-03-10 09:08:04.948', 'TerryJ', 100)
('2024-03-10 09:08:04.955', 'TerryJ', 5)
('2024-03-10 09:08:04.961', 'Graham', -100)
('2024-03-10 09:08:04.968', 'Sally', 1000)
('2024-03-10 09:08:04.974', 'TerryG', 4000)
('2024-03-10 09:08:17.918', 'John', 2002)
('2024-03-10 09:08:17.930', 'John', -1010)
('2024-03-10 09:08:17.938', 'Terry', 1)
('2024-03-10 09:08:17.946', 

**The SQLite `strftime()` function converts the time field into a string, with the format specified for the date and time, the column containing the time values, and the modifier 'localtime', which converts UTC time to localtime.**

**Note that the time function truncates the number of milliseconds to three decimal points, because you used `'%f'` format.**

**You can create a 'view' of the transactions with the converted time field:**

In [12]:
db.execute("CREATE VIEW IF NOT EXISTS history AS SELECT strftime('%Y-%m-%d %H:%M:%f', transactions.time, 'localtime') AS localtime,"
          " transactions.account, transactions.amount FROM transactions ORDER BY transactions.time")

In [13]:
for row in db.execute("SELECT * FROM history"):
    print(row)

('2024-03-10 09:03:06.514', 'John', 20.02)
('2024-03-10 09:03:06.521', 'John', -10.1)
('2024-03-10 09:03:06.526', 'Terry', 0.1)
('2024-03-10 09:03:06.532', 'Terry', 0.1)
('2024-03-10 09:03:06.539', 'Terry', 0.1)
('2024-03-10 09:03:06.550', 'TerryJ', 100)
('2024-03-10 09:03:06.555', 'TerryJ', 0.5)
('2024-03-10 09:03:06.561', 'Graham', -100)
('2024-03-10 09:03:06.566', 'Sally', 1000)
('2024-03-10 09:03:06.576', 'TerryG', 40)
('2024-03-10 09:08:04.912', 'John', 2002)
('2024-03-10 09:08:04.921', 'John', -1010)
('2024-03-10 09:08:04.927', 'Terry', 1)
('2024-03-10 09:08:04.934', 'Terry', 1)
('2024-03-10 09:08:04.941', 'Terry', 1)
('2024-03-10 09:08:04.948', 'TerryJ', 100)
('2024-03-10 09:08:04.955', 'TerryJ', 5)
('2024-03-10 09:08:04.961', 'Graham', -100)
('2024-03-10 09:08:04.968', 'Sally', 1000)
('2024-03-10 09:08:04.974', 'TerryG', 4000)
('2024-03-10 09:08:17.918', 'John', 2002)
('2024-03-10 09:08:17.930', 'John', -1010)
('2024-03-10 09:08:17.938', 'Terry', 1)
('2024-03-10 09:08:17.946', 

In [14]:
db.close()

**In the `Account` class, the transaction times are stored as UTC date and time, but you can store the local times also. It is best to add an extra column to the transaction table, for timezones or offset values or the local times (as strings).**

In [15]:
db = sqlite3.connect('accounts/accounts_timezone.sqlite', detect_types=sqlite3.PARSE_DECLTYPES)

db.execute("CREATE TABLE IF NOT EXISTS accounts (name TEXT PRIMARY KEY NOT NULL, balance INTEGER NOT NULL)")
db.execute("CREATE TABLE IF NOT EXISTS transactions (time TIMESTAMP NOT NULL, timezone INTEGER NOT NULL, "
           "account TEXT NOT NULL, amount INTEGER NOT NULL, PRIMARY KEY (time, account))")

In [16]:
import pickle

# _current_time() and _save_update() methods updated
class Account(object):
    
    @staticmethod
    def _current_time():
        utc_time = pytz.utc.localize(datetime.datetime.utcnow())
        local_time = utc_time.astimezone()
        timezone = local_time.tzinfo
        return utc_time, timezone
    
    def __init__(self, name, opening_balance=0):
        cursor = db.execute("SELECT name, balance FROM accounts WHERE (name=?)", (name, ))
        row = cursor.fetchone()
        
        if row:
            self.name, self._balance = row
            print(f"\tRetrieved account details for {self.name}")
        else:
            self.name = name
            self._balance = opening_balance
            cursor.execute("INSERT INTO accounts VALUES(?, ?)", (name, opening_balance))
            cursor.connection.commit()
            print(f"\tAccount created for {self.name}")
            
        self.show_balance()
    
    def _save_update(self, amount):
        new_balance = self._balance + amount
        deposit_time, deposit_zone = Account._current_time()
        # Pickle the timezone
        pickle_timezone = pickle.dumps(deposit_zone)
        db.execute("UPDATE accounts SET balance= ? WHERE (name=?)", (new_balance, self.name))
        db.execute("INSERT INTO transactions VALUES(?, ?, ?, ?)", (deposit_time, pickle_timezone, self.name, amount))
        db.commit()
        self._balance = new_balance
    
    def deposit(self, amount):
        if amount > 0.0:
            self._save_update(amount)
            print("£{:.2f} deposited".format(amount / 100))
            
        return self._balance / 100
    
    def withdraw(self, amount):
        if 0.0 < amount <= self._balance:
            self._save_update(-amount)
            print("£{:.2f} withdrawn".format(amount / 100))
            return amount / 100
        else:
            print(f"Not enough funds to withdraw £{amount}")
            return 0.0
    
    def show_balance(self):
        print("Balance on account is £{:.2f}".format(self._balance / 100))


In [17]:
john = Account('John')
john.deposit(2002)
john.withdraw(1010)
john.show_balance()

print('*' * 80)

terry = Account('Terry')
terry.deposit(1)
terry.deposit(1)
terry.deposit(1)
terry.show_balance()

print('*' * 80)

# New account
terryJ = Account('TerryJ')
terryJ.deposit(100)
terryJ.deposit(5)
terryJ.show_balance()

print('*' * 80)

graham = Account('Graham')
graham.withdraw(100)
graham.show_balance()

print('*' * 80)

sally = Account('Sally')
sally.deposit(1000)
sally.show_balance()

print('*' * 80)

# New account
terryG = Account('TerryG')
terryG.deposit(4000)
terryG.withdraw(5000)
terryG.show_balance()

print('*' * 80)

	Account created for John
Balance on account is £0.00
£20.02 deposited
£10.10 withdrawn
Balance on account is £9.92
********************************************************************************
	Account created for Terry
Balance on account is £0.00
£0.01 deposited
£0.01 deposited
£0.01 deposited
Balance on account is £0.03
********************************************************************************
	Account created for TerryJ
Balance on account is £0.00
£1.00 deposited
£0.05 deposited
Balance on account is £1.05
********************************************************************************
	Account created for Graham
Balance on account is £0.00
Not enough funds to withdraw £100
Balance on account is £0.00
********************************************************************************
	Account created for Sally
Balance on account is £0.00
£10.00 deposited
Balance on account is £10.00
********************************************************************************
	Account crea

In [18]:
for row in db.execute("SELECT * FROM transactions"):
    print(row)

(datetime.datetime(2024, 3, 12, 7, 30, 24, 255212), b'\x80\x04\x95K\x00\x00\x00\x00\x00\x00\x00\x8c\x08datetime\x94\x8c\x08timezone\x94\x93\x94h\x00\x8c\ttimedelta\x94\x93\x94K\x00K\x00K\x00\x87\x94R\x94\x8c\x11GMT Standard Time\x94\x86\x94R\x94.', 'John', 2002)
(datetime.datetime(2024, 3, 12, 7, 30, 24, 266068), b'\x80\x04\x95K\x00\x00\x00\x00\x00\x00\x00\x8c\x08datetime\x94\x8c\x08timezone\x94\x93\x94h\x00\x8c\ttimedelta\x94\x93\x94K\x00K\x00K\x00\x87\x94R\x94\x8c\x11GMT Standard Time\x94\x86\x94R\x94.', 'John', -1010)
(datetime.datetime(2024, 3, 12, 7, 30, 24, 282683), b'\x80\x04\x95K\x00\x00\x00\x00\x00\x00\x00\x8c\x08datetime\x94\x8c\x08timezone\x94\x93\x94h\x00\x8c\ttimedelta\x94\x93\x94K\x00K\x00K\x00\x87\x94R\x94\x8c\x11GMT Standard Time\x94\x86\x94R\x94.', 'Terry', 1)
(datetime.datetime(2024, 3, 12, 7, 30, 24, 289644), b'\x80\x04\x95K\x00\x00\x00\x00\x00\x00\x00\x8c\x08datetime\x94\x8c\x08timezone\x94\x93\x94h\x00\x8c\ttimedelta\x94\x93\x94K\x00K\x00K\x00\x87\x94R\x94\x8c\x11G

**The timezone info object is a bytes literal that is really long... It looks weird and decoded, it does not reveal anything because it has been 'pickled'.**

**You 'load' (opposite of dumping it) the pickled tzinfo object to then generate the local time (which in your case is the same time).**

In [35]:
for row in db.execute("SELECT * FROM transactions"):
    utc_time = row[0]
    pickled_zone = row[1]
    zone = pickle.loads(pickled_zone)
    local_time = pytz.utc.localize(utc_time).astimezone(zone)
    print(f"UTC: {utc_time}\t\tLOCAL TIME: {local_time}\tTIMEZONE: {local_time.tzinfo}")

UTC: 2024-03-12 07:30:24.255212		LOCAL TIME: 2024-03-12 07:30:24.255212+00:00	TIMEZONE: GMT Standard Time
UTC: 2024-03-12 07:30:24.266068		LOCAL TIME: 2024-03-12 07:30:24.266068+00:00	TIMEZONE: GMT Standard Time
UTC: 2024-03-12 07:30:24.282683		LOCAL TIME: 2024-03-12 07:30:24.282683+00:00	TIMEZONE: GMT Standard Time
UTC: 2024-03-12 07:30:24.289644		LOCAL TIME: 2024-03-12 07:30:24.289644+00:00	TIMEZONE: GMT Standard Time
UTC: 2024-03-12 07:30:24.297189		LOCAL TIME: 2024-03-12 07:30:24.297189+00:00	TIMEZONE: GMT Standard Time
UTC: 2024-03-12 07:30:24.311585		LOCAL TIME: 2024-03-12 07:30:24.311585+00:00	TIMEZONE: GMT Standard Time
UTC: 2024-03-12 07:30:24.317961		LOCAL TIME: 2024-03-12 07:30:24.317961+00:00	TIMEZONE: GMT Standard Time
UTC: 2024-03-12 07:30:24.338252		LOCAL TIME: 2024-03-12 07:30:24.338252+00:00	TIMEZONE: GMT Standard Time
UTC: 2024-03-12 07:30:24.354807		LOCAL TIME: 2024-03-12 07:30:24.354807+00:00	TIMEZONE: GMT Standard Time


In [36]:
db.close()